In [18]:
from google.colab import drive
import os
import glob
import pandas as pd

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/data-science-veiculos-poluicao'
os.chdir(folder_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
estacoes_iema = pd.read_csv('iema-agg/estacoes-iema.csv')

display(estacoes_iema.head())

,Estação,Município1,Estação1,Estado,Sigla Da Região,Município (Estações),Código Estação Iema
0,Centro Administrativo,Betim,Centro Administrativo,MG,RM-BH,Betim,MG07
1,Araçatuba,Araçatuba,Araçatuba,SP,Interior,Araçatuba,SP02
2,Rs - Casa da Lua,Resende,Rs - Casa da Lua,RJ,RMP,Resende,RJ54
3,Canoas - V COMAR,Canoas,Canoas - V COMAR,RS,RM-POA,Canoas,RS01
4,Cerqueira César,São Paulo,Cerqueira César,SP,RM-SP,São Paulo,SP14


,UF,Município
0,BAHIA,ABAIRA
1,BAHIA,ABARE
2,BAHIA,ACAJUTIBA
3,BAHIA,AGUA FRIA
4,BAHIA,AIQUARA


In [20]:
estacoes_iema = estacoes_iema[["Município (Estações)", "Código Estação Iema"]]
estacoes_iema = estacoes_iema.rename(columns={"Município (Estações)": "city", "Código Estação Iema": "id"})
display(estacoes_iema.head())

,city,id
0,Betim,MG07
1,Araçatuba,SP02
2,Resende,RJ54
3,Canoas,RS01
4,São Paulo,SP14


In [21]:
import unicodedata

def remove_accents(text):
    """Removes accent characters from a string."""
    if isinstance(text, str):
        return ''.join(
            char for char in unicodedata.normalize('NFD', text)
            if unicodedata.category(char) != 'Mn'
        )
    return text

estacoes_iema['city'] = estacoes_iema['city'].apply(remove_accents).str.upper()

display(estacoes_iema.head())

,city,id
0,BETIM,MG07
1,ARACATUBA,SP02
2,RESENDE,RJ54
3,CANOAS,RS01
4,SAO PAULO,SP14


In [22]:
initial_rows = estacoes_iema.shape[0]
estacoes_iema.drop_duplicates(subset=['city', 'id'], inplace=True)
rows_after_dropping_duplicates = estacoes_iema.shape[0]
rows_deleted = initial_rows - rows_after_dropping_duplicates

print(f"Initial number of rows: {initial_rows}")
print(f"Number of rows after dropping duplicates: {rows_after_dropping_duplicates}")
print(f"Number of rows deleted: {rows_deleted}")

display(estacoes_iema.head())

Initial number of rows: 2197
Number of rows after dropping duplicates: 260
Number of rows deleted: 1937


,city,id
0,BETIM,MG07
1,ARACATUBA,SP02
2,RESENDE,RJ54
3,CANOAS,RS01
4,SAO PAULO,SP14


In [25]:
df_uf_municipio = pd.read_csv('senatran-agg/uf_municipio.csv')
display(df_uf_municipio.head())

,UF,Município
0,BAHIA,ABAIRA
1,BAHIA,ABARE
2,BAHIA,ACAJUTIBA
3,BAHIA,AGUA FRIA
4,BAHIA,AIQUARA


In [28]:
# Rename the 'Município' column to 'cidade' in df_uf_municipio
df_uf_municipio = df_uf_municipio.rename(columns={'Município': 'cidade'})

# Get the unique cities from estacoes_iema
estacoes_iema_cities = estacoes_iema['cidade'].unique()

# Get the unique cities from df_uf_municipio
uf_municipio_cities = df_uf_municipio['cidade'].unique()

# Find cities in estacoes_iema that are not in df_uf_municipio
cities_not_in_uf_municipio = set(estacoes_iema_cities) - set(uf_municipio_cities)

print("Cities in estacoes_iema not found in df_uf_municipio:")
print(cities_not_in_uf_municipio)

# Display the head of the modified df_uf_municipio
display(df_uf_municipio.head())

Cities in estacoes_iema not found in df_uf_municipio:
{'RETIRO', 'SAO JOAO DO MERITI'}


,UF,cidade
0,BAHIA,ABAIRA
1,BAHIA,ABARE
2,BAHIA,ACAJUTIBA
3,BAHIA,AGUA FRIA
4,BAHIA,AIQUARA


In [29]:
# Remove 'RETIRO' from the set of cities not in df_uf_municipio
cities_not_in_uf_municipio.discard('RETIRO')

# Replace 'SAO JOAO DO MERITI' with 'SAO JOAO DE MERITI' in the 'cidade' column of estacoes_iema
estacoes_iema['cidade'] = estacoes_iema['cidade'].replace('SAO JOAO DO MERITI', 'SAO JOAO DE MERITI')

# Verify the changes by printing the updated set and checking the dataframe
print("Updated cities not in df_uf_municipio set:")
print(cities_not_in_uf_municipio)

print("\nChecking for 'SAO JOAO DO MERITI' in estacoes_iema after replacement:")
display(estacoes_iema[estacoes_iema['cidade'] == 'SAO JOAO DO MERITI'])

print("\nChecking for 'SAO JOAO DE MERITI' in estacoes_iema after replacement:")
display(estacoes_iema[estacoes_iema['cidade'] == 'SAO JOAO DE MERITI'])

Updated cities not in df_uf_municipio set:
{'SAO JOAO DO MERITI'}

Checking for 'SAO JOAO DO MERITI' in estacoes_iema after replacement:


,cidade,id



Checking for 'SAO JOAO DE MERITI' in estacoes_iema after replacement:


,cidade,id
38,SAO JOAO DE MERITI,RJ44


In [30]:
# Perform an inner join between estacoes_iema and df_uf_municipio on the 'cidade' column
estacoes_municipios_joined = pd.merge(estacoes_iema, df_uf_municipio, on='cidade', how='inner')

print("\nHead of the joined dataframe:")
display(estacoes_municipios_joined.head())

print("\nShape of the joined dataframe:")
print(estacoes_municipios_joined.shape)


Head of the joined dataframe:


,cidade,id,UF
0,BETIM,MG07,MINAS GERAIS
1,ARACATUBA,SP02,SAO PAULO
2,RESENDE,RJ54,RIO DE JANEIRO
3,CANOAS,RS01,RIO GRANDE DO SUL
4,SAO PAULO,SP14,SAO PAULO



Shape of the joined dataframe:
(274, 3)


In [33]:
# Define the rows to be deleted as a list of dictionaries
rows_to_delete_list = [
    {'cidade': 'ANCHIETA', 'id': 'ES16', 'UF': 'SANTA CATARINA'},
    {'cidade': 'SANTO ANDRE', 'id': 'SP61', 'UF': 'PARAIBA'},
    {'cidade': 'CANDEIAS', 'id': 'BA10', 'UF': 'MINAS GERAIS'},
    {'cidade': 'CANTAGALO', 'id': 'RJ78', 'UF': 'MINAS GERAIS'},
    {'cidade': 'CANTAGALO', 'id': 'RJ78', 'UF': 'PARANA'},
    {'cidade': 'CANTAGALO', 'id': 'RJ77', 'UF': 'MINAS GERAIS'},
    {'cidade': 'CANTAGALO', 'id': 'RJ77', 'UF': 'PARANA'},
    {'cidade': 'SANTO ANDRE', 'id': 'SP62', 'UF': 'PARAIBA'},
    {'cidade': 'ANCHIETA', 'id': 'ES13', 'UF': 'SANTA CATARINA'},
    {'cidade': 'SAO SEBASTIAO', 'id': 'SP145', 'UF': 'ALAGOAS'},
    {'cidade': 'ANCHIETA', 'id': 'ES15', 'UF': 'SANTA CATARINA'},
    {'cidade': 'ANCHIETA', 'id': 'ES17', 'UF': 'SANTA CATARINA'},
    {'cidade': 'ANCHIETA', 'id': 'ES12', 'UF': 'SANTA CATARINA'},
    {'cidade': 'RIO CLARO', 'id': 'SP54', 'UF': 'RIO DE JANEIRO'}
]

# Create a boolean mask to identify rows to delete
mask_to_delete = pd.Series(False, index=estacoes_municipios_joined.index)
for row in rows_to_delete_list:
    mask_to_delete |= (
        (estacoes_municipios_joined['cidade'] == row['cidade']) &
        (estacoes_municipios_joined['id'] == row['id']) &
        (estacoes_municipios_joined['UF'] == row['UF'])
    )

# Delete the rows using the boolean mask
estacoes_municipios_cleaned = estacoes_municipios_joined[~mask_to_delete].copy()

print("Original shape:", estacoes_municipios_joined.shape)
print("Shape after deleting rows:", estacoes_municipios_cleaned.shape)

# Display the head of the cleaned dataframe
display(estacoes_municipios_cleaned.head())

Original shape: (274, 3)
Shape after deleting rows: (260, 3)


,cidade,id,UF
0,BETIM,MG07,MINAS GERAIS
1,ARACATUBA,SP02,SAO PAULO
2,RESENDE,RJ54,RIO DE JANEIRO
3,CANOAS,RS01,RIO GRANDE DO SUL
4,SAO PAULO,SP14,SAO PAULO


In [34]:
# Save the joined dataframe to a CSV file in Google Drive
output_path = os.path.join(folder_path, 'cidades_lookup.csv')
estacoes_municipios_joined.to_csv(output_path, index=False)

print(f"DataFrame saved to: {output_path}")

DataFrame saved to: /content/drive/MyDrive/data-science-veiculos-poluicao/cidades_lookup.csv
